In [4]:
import pandas as pd
import tabula
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import time
import random

In [5]:
def busca_cofepris(medicamento):
    #función para cambiar la denominación del medicamento
    def denominacion(option_text):
        combo = driver.find_element(By.CLASS_NAME, "form-control")
        combo_select = Select(combo)
        combo_select.select_by_visible_text(option_text)
        cuadro_busqueda = driver.find_element(By.ID, "MainContent_txtSearchValue")
        cuadro_busqueda.send_keys(medicamento)
        button = driver.find_element(By.XPATH, "//button[contains(@onclick, '__doPostBack') and @class='btn btn-primary']")
        button.click()
    #Función para extraer los elementos de la tabla
    def elementos_tabla(driver):
        table_element = driver.find_element(By.CLASS_NAME, "gridView_basic")
        table_contents = table_element.get_attribute('outerHTML')
        return table_contents
    
    #Código para extraer los datos de la página
    url = "https://tramiteselectronicos02.cofepris.gob.mx/BuscadorPublicoRegistrosSanitarios/BusquedaRegistroSanitario.aspx"
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-cache')
    options.add_argument('--disable-cookies')
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service("C:/chromedriver.exe"), options=options)
    driver.get(url)
    #Denominación Distintiva
    denominacion("Denominación Distintiva")

    try:
        table_contents = elementos_tabla(driver)
    except NoSuchElementException:
        #Denominación Genérica
        denominacion("Denominación Genérica")
        try:
            table_contents = elementos_tabla(driver)
        except NoSuchElementException:
            driver.quit()
            return pd.DataFrame()

    soup = BeautifulSoup(table_contents, 'html.parser')
    registro=[]
    marca = []
    generica = []
    indica = []
    lab = []
    fabricante = []
    principio=[]

    for span in soup.find_all('span'):
        if span.has_attr('id'):
            if span['id'].startswith('MainContent_GV_RegistroSanitario_lblNumeroRegistro'):
                registro.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionDistintiva'):
                marca.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionGenerica'):
                generica.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblTitularRegistro'):
                lab.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblFabricanteMedicamento'):
                fabricante.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblPrincipioActivo'):
                principio.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblIndicacionTerapeutica'):
                indica.append(span.text)

    df = pd.DataFrame({'reg_san':registro,'marca': marca, 'den_gen': generica, 'titular': lab, 'principio': principio,'indicacion':indica,"busqueda":medicamento})
    df = df.apply(lambda x: x.astype(str).str.lower())
    df = df.apply(lambda x: x.astype(str).str.replace('\n', ''))
    driver.quit()
    return df


In [6]:
busca_cofepris("fluticasona")

,reg_san,marca,den_gen,titular,principio,indicacion,busqueda
0,106m2019 ssa,flofera,fluticasona / formoterol,,r03ak11 formoterol y fluticasone,tratamiento del asma moderada a asma severa pe...,fluticasona
1,382m2015 ssa,xygalbiot,fluticasona / salmeterol,"laboratorios elea mexico, s.a. de c.v.",r03ak06 salmeterol y fluticasone,"broncodilatador, antinflamatorio esteroideo.",fluticasona
2,052m2016 ssa,nimbus,fluticasona / salmeterol,"laboratorios liomont,s.a.de c.v.",r03ba05 fluticasona,"broncodilatador, enfermedad pulmonar obstructi...",fluticasona
3,374m2017 ssa,trelegy,fluticasona / umeclidinio / vilanterol,,r03bx99 otros principios activos. otros agent...,tratamiento de mantenimiento para prevenir y a...,fluticasona
4,331m2014 ssa,innube,fluticasona,,r01ad08 fluticasona,tratamiento y profilaxis de la rinitis alérgic...,fluticasona
